In [1]:
import ir_datasets
import pandas as pd
import pyterrier as pt

if not pt.started():
    pt.init()


C:\Users\yacin\AppData\Local\Temp\ipykernel_45656\738761206.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\yacin\AppData\Local\Temp\ipykernel_45656\738761206.py:6: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [2]:
# corpus
dataset = ir_datasets.load("cranfield")

docs = [{
    "docno": doc.doc_id,
    "text": doc.text
} for doc in dataset.docs]

df_docs = pd.DataFrame(docs)
# Nettoyage robuste
import re


df_docs = df_docs.dropna(subset=["text", "docno"])
df_docs = df_docs[df_docs["text"].str.strip() != ""]
df_docs = df_docs[df_docs["docno"].str.strip() != ""]
df_docs = df_docs[df_docs["text"].apply(lambda x: bool(re.search(r"\w{3,}", x)))]
df_docs = df_docs.drop_duplicates(subset="docno")
display(df_docs.head())
df_docs.to_csv("cranfield_docs.csv", index=False)

,docno,text
0,1,experimental investigation of the aerodynamics...
1,2,simple shear flow past a flat plate in an inco...
2,3,the boundary layer in simple shear flow past a...
3,4,approximate solutions of the incompressible la...
4,5,one-dimensional transient heat conduction into...


In [3]:
# queries
topics = [{
    "qid": topic.query_id,
    "query": topic.text
} for topic in dataset.queries]

df_topics = pd.DataFrame(topics)
import re
def clean_query(query):
    return re.sub(r'/', ' ', query)

df_topics['query'] = df_topics['query'].apply(clean_query)
df_topics = df_topics.dropna(subset=["query", "qid"])
df_topics = df_topics[df_topics["query"].str.strip() != ""]
df_topics = df_topics[df_topics["qid"].str.strip() != ""]

display(df_topics.head())
df_topics.to_csv("cranfield_topics.csv", index=False)

,qid,query
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,3,what problems of heat conduction in composite ...
3,4,can a criterion be developed to show empirical...
4,5,what chemical kinetic system is applicable to ...


In [4]:
# qrels
qrels = [{
    "qid": qrel.query_id,
    "docno": qrel.doc_id,
    "label": qrel.relevance
} for qrel in dataset.qrels]

df_qrels = pd.DataFrame(qrels)
display(df_qrels.head())
df_qrels.to_csv("cranfield_qrels.csv", index=False)

,qid,docno,label
0,1,184,2
1,1,29,2
2,1,31,2
3,1,12,3
4,1,51,3


In [6]:
import os

index_path = os.path.abspath("my_index")
index_path = os.path.join(os.getcwd(), index_path)
#delete the direction if path exists
os.makedirs(index_path, exist_ok=True)

indexer = pt.IterDictIndexer(index_path, meta=["docno", "text"], overwrite=True)
index_ref = indexer.index(
    ({"docno": str(row["docno"]), "text": row["text"]} for _, row in df_docs.iterrows())
)

c:\Users\yacin\Desktop\M1\RI\M1-S2-RI\.conda\Lib\site-packages\pyterrier\terrier\index.py:639: UserWarning: Indexing meta key text length requested 512 but exceeded in first document (actual length 910). Increase the length in the meta dict for the indexer, e.g., pt.IterDictIndexer(..., meta={'text': 910}).
  warn(msg)


AttributeError: 'TfidfVectorizer' object has no attribute 'save'

: 